In [1]:
import re
from dateparser.search import search_dates
import os
import pandas as pd
import import_ipynb
from Tesseract import *

importing Jupyter notebook from Tesseract.ipynb


In [2]:
DataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")

In [3]:
def get_invoice_nums(all_words):
    inv_nums = []
    invoice_no_re = r'^[0-9a-zA-Z-:/]+$'
    for word in all_words:
        if not re.search('\d', word['text']):
            continue
        result = re.findall(invoice_no_re,word['text'])
        if result:
            inv_nums.append({
                'text': word['text'],
                'x1': word['left'],
                'y1': word['top'],
                'x2': word['left'] + word['width'],
                'y2': word['top'] + word['height']
            })

    return inv_nums

In [4]:
def get_amounts(all_words):
    amounts = []
    amount_re = r"^\$?\d+(,\d{3})*(\.\d+)?$"
    for word in all_words:
        if not re.search(amount_re, word['text']):
            continue
        if len(word)<3:
            continue
        result = re.findall(amount_re,word['text'])
        if result:
            amounts.append({
                'text': word['text'],
                'x1': word['left'],
                'y1': word['top'],
                'x2': word['left'] + word['width'],
                'y2': word['top'] + word['height']
            })

    return amounts

In [28]:
def get_dates(all_text, all_words):
    dates, all_dates = [], []
    indices = []
    matches = search_dates(all_text)

    
    for match in matches:
        text = match[0]

        token_length = len(text.split(' '))
        idx = all_text.find(match[0])
        text_len = len(text)
        index = len(all_text[:idx].strip().split(' '))

        replaced_text = ' '.join(['*'*len(i) for i in text.split(' ')])

        indices.append(list(range(index, index + token_length)))

        index += token_length
        all_text = all_text[:idx + text_len].replace(text, replaced_text) + all_text[idx + text_len:]
    
    for date_indices in indices:
        date = ''
        left, top, right, bottom = [], [], [], []
        
        for i in date_indices:
            if i<len(all_words):
                date += ' ' + all_words[i]['text']
                left.append(all_words[i]['left'])
                top.append(all_words[i]['top'])
                right.append(all_words[i]['left'] + all_words[i]['width'])
                bottom.append(all_words[i]['top'] + all_words[i]['height'])
        all_dates.append({
            'text': date.strip(),
            'x1': min(left),
            'y1': min(top),
            'x2': max(right),
            'y2': max(bottom)
        })

    return all_dates


In [29]:
# def get_dates(all_words):
#     amounts = []
#     amount_re = r'\b((\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4})|(\d{1,2}[-/.]\d{1,2}[-/.]\d{2})|(\d{4}[-/.]\d{1,2}[-/.]\d{1,2})|(\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{4})|((?:Mon|Tue|Wed|Thu|Fri|Sat|Sun)[a-z]*,\s\d{1,2}(?:st|nd|rd|th)\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*))\b'

#     for word in all_words:
#         if not re.search(amount_re, word['text']):
#             continue
#         if len(word)<3:
#             continue
#         result = re.findall(amount_re,word['text'])
#         if result:
#             amounts.append({
#                 'text': word['text'],
#                 'x1': word['left'],
#                 'y1': word['top'],
#                 'x2': word['left'] + word['width'],
#                 'y2': word['top'] + word['height']
#             })

#     return amounts

In [32]:
def get_candidates(file):
        image = cv2.imread(file)
        data = ocr(file)
        data["left"]/=image.shape[1]
        data["top"]/=image.shape[0]
        data["width"]/=image.shape[1]
        data["height"]/=image.shape[0]
        data["X"]/=image.shape[1]
        data["Y"]/=image.shape[0]

        all_words = []
        for idx in data.index:
            word = data["text"][idx]
            if word.strip() != "":
                all_words.append({
                    'text': data['text'][idx],
                    'left': data['left'][idx],
                    'top': data['top'][idx],
                    'width': data['width'][idx],
                    'height': data['height'][idx]})
        text = ' '.join([word['text'].strip() for word in all_words])

        invoice_date_candidates = get_dates(text, all_words)
    
        total_amount_candidates = get_amounts(all_words)
    
        invoice_no_candidates = get_invoice_nums(all_words)

        candidate_data = {
            'invoice_no': invoice_no_candidates,
            'invoice_date': invoice_date_candidates,
            'total_amount': total_amount_candidates
        }
        Dataframes = []
        for key in candidate_data.keys():
            df = pd.DataFrame(candidate_data[key])
            df["Class"] = key
            Dataframes.append(df)
        return data, Dataframes

In [33]:
D, DF = get_candidates("2.jpeg")

In [34]:
DF[1]

,text,x1,y1,x2,y2,Class
0,22-Sep-22,0.151674,0.138699,0.217023,0.148402,invoice_date
1,"floor,,",0.497378,0.199201,0.534893,0.208333,invoice_date
2,"29,",0.674466,0.212614,0.694232,0.221461,invoice_date
3,19-Sep-22,0.726906,0.282534,0.799516,0.292523,invoice_date
4,any,0.599032,0.464326,0.619605,0.470034,invoice_date
5,@ 24%,0.536910,0.479737,0.575232,0.486301,invoice_date
6,1-Oct-22,0.664784,0.554509,0.718031,0.561358,invoice_date
7,"31 1,98,603.00",0.743445,0.552511,0.905607,0.561644,invoice_date
8,-Oct-22,0.664381,0.566495,0.709964,0.573345,invoice_date
9,1,0.102864,0.579909,0.108511,0.585046,invoice_date
